In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import copy
import pandas as pd
%matplotlib inline
import pyram as pr

Welcome to pyram: your Raman analysis library!


In [2]:
pr.cwd('example_data')
pr.charge_lib('short')

Charging pure spectra library, needs around a minute ....
Done. Good work!


In [7]:
lib=pr.lib()
lib_names=list(lib.keys())

In [195]:
pr.set_alias()
S=pr.type2spectra('txt8')
S=pr.bkg_subtraction(S,sigma=100,plot=True)
pr.plot(S)


txt0  ->  Sample01_A_10x6s_633nm_G1200_F100__Ox100LWD.txt
txt1  ->  Sample01_B_10x30s_633nm_G1200_F1__Ox100LWD.txt
txt2  ->  Sample01_C_10x20s_633nm_G1200_F10__Ox100LWD.txt
txt3  ->  Sample01_D_6x4s_633nm_G1200_F100__Ox10.txt
txt4  ->  Sample01_E_6x4s_633nm_G1200_F100__Ox100LWD.txt
txt5  ->  Sample01_F_10x5s_633nm_G1200_F0_1__Ox100LWD.txt
txt6  ->  Sample01_G_6x10s_633nm_G1200_F10__Ox100LWD.txt
txt7  ->  Sample01_H_10x10s_633nm_G1200_F10__Ox100LWD.txt
txt8  ->  Sample01_I_10x10s_633nm_G1200_F25__Ox100LWD.txt
txt9  ->  Sample01_J_10x5s_633nm_G1200_F10__Ox100LWD.txt
txt10  ->  Sample01_K_10x10s_633nm_G1200_F10__Ox100LWD.txt
txt11  ->  Si_calibration_A_60x1s_633nmnm_G1200_F100__Ox100.txt
sum(Y-XW) =  -18111869.616152495
message =  Optimization terminated successfully.
# of iter =  2303


In [196]:
shift=10

match = []
names = []
step = np.linspace(0,1,5)
step = step[1:-1]
for a in range(len(lib_names)):
    for b in range(len(lib_names)):
        if b>a:
            
            B = [0]*2
            N = 2
            B[0]=pr.lib(lib_names[a]).copy()
            B[0][1] /=  np.max(B[0][1])
            B[1]=pr.lib(lib_names[b]).copy()
            B[1][1] /=  np.max(B[1][1])
            min = np.max([B[0][0,0],B[1][0,0]])
            max = np.min([B[0][0,-1],B[1][0,-1]])
            min_A = np.where(B[0][0]==min)[0][0]
            max_A = np.where(B[0][0]==max)[0][0]
            min_B = np.where(B[1][0]==min)[0][0]
            max_B = np.where(B[1][0]==max)[0][0]
            B[0] = B[0][:,min_A:max_A]
            B[1] = B[1][:,min_B:max_B]

            temp=[0,0]
            for i in step:
                X = np.array([B[0][0,:], i*B[0][1,:]+(1-i)*B[1][1,:]])
                t = pr.cos_sim_w_shift(S,X ,shift)
                if t>temp[1]:
                    temp[1] = t
                    temp[0] = str(i)+' '+lib_names[a]+', '+str(1-i)+' '+lib_names[b]
                

            names.append(temp[0])
            match.append(temp[1])
        
        if b==a:
            B = pr.lib(lib_names[a]).copy()
            names.append(lib_names[a])
            match.append(pr.cos_sim_w_shift(S,B,shift))
            

match = pd.DataFrame({'spectra':names, 'match':match})    
match.sort_values('match', ascending=False, inplace=True)
match.reset_index(inplace=True, drop=True)
print(match.head(10))

                           spectra     match
0      0.75 Albite, 0.25 Muscovite  0.865801
1    0.75 Albite, 0.25 Armalcolite  0.859376
2   0.75 Albite, 0.25 Chalcopyrite  0.857216
3     0.75 Albite, 0.25 Nontronite  0.856793
4   0.75 Albite, 0.25 Pyrophyllite  0.852844
5       0.75 Albite, 0.25 Pimelite  0.847900
6     0.75 Albite, 0.25 Phlogopite  0.845540
7   0.75 Albite, 0.25 Arsenopyrite  0.841883
8  0.75 Albite, 0.25 Lepidocrocite  0.841174
9        0.75 Albite, 0.25 Amesite  0.841024


In [212]:
pr.search(S,shift=10)

  alias          name     match
0  sch0        Albite  0.835516
1  sch1    Microcline  0.661255
2  sch2    Orthoclase  0.603206
3  sch3    Nontronite  0.578685
4  sch4  Chalcopyrite  0.574209
5  sch5       Zoisite  0.470706
6  sch6  Arsenopyrite  0.442060
7  sch7       Amesite  0.438392
8  sch8   Armalcolite  0.433378
9  sch9    Dwornikite  0.432447


In [155]:
%matplotlib qt

In [193]:
A = pr.lib('Albite')
B = pr.lib('Armalcolite')

pr.plot([A,S])